# Convergence of IBCM and BioPCA habituation to turbulent backgrounds

# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os, json
pj = os.path.join

# Initialization

### Aesthetic parameters

In [ ]:
do_save_plots = True
# Resources
root_dir = pj("..", "..", "..")
data_folder = pj(root_dir, "results", "for_plots")
data_folder_conv = pj(root_dir, "results", "for_plots", "convergence")
panels_folder = "panels/"
params_folder = pj(root_dir, "results", "common_params")

In [ ]:
# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full = np.asarray(json.load(f))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

In [ ]:
# Extra aesthetic parameters for this figure
# Figures slightly less high, to squeeze four rows of plots
plt.rcParams["figure.figsize"] = (plt.rcParams["figure.figsize"][0], 1.6)

# More legend rcParams: make everything smaller by 30 %
plt.rcParams["patch.linewidth"] = 0.75
legend_rc = {"labelspacing":0.5, "handlelength":2.0, "handleheight":0.7, 
             "handletextpad":0.8, "borderaxespad":0.5, "columnspacing":2.0}
for k in legend_rc:
    plt.rcParams["legend."+k] = 0.75 * legend_rc[k]

new_color = "r"
linestyles = ["-", "--", ":", (0, (5, 1, 2, 1)), "-."]
neuron_styles = linestyles + [(0, (1, 2, 1, 2))]

markerstyles = ["o", "s", "^", "v", "X", "*", "d", "h", "<", "p", "P"]

# IBCM plotting functions

In [ ]:
def plot_ibcm_hgammas_series(t_axis, i_highlights, hgammaser, squeeze=0.65):
    # Show three neurons
    fig, ax = plt.subplots()
    # By default, we squeeze to be able to put matrix of cgammas series besides
    fig.set_size_inches(plt.rcParams["figure.figsize"][0]*squeeze, 
                        plt.rcParams["figure.figsize"][1])

    #ax.axhline(0.0, ls="-", color=(0.8,)*3, lw=0.8)
    legend_styles = [[0,]*6, [0,]*6, [0,]*6]
    neuron_colors3 = neuron_colors_full[[8, 17, 23]]
    clr_back = back_palette[-1]
    plot_skp = 20
    n_b = hgammaser.shape[2]

    # plot all other neurons first, skip some points
    for i in range(n_i_ibcm):
        if i in i_highlights: 
            continue
        elif i % 2 == 0:   # thinning
            continue
        else: 
            for j in range(n_b):
                ax.plot(t_axis[::plot_skp], hgammaser[::plot_skp, i, j], color=clr_back, 
                    ls="-", alpha=1.0-0.1*j, lw=plt.rcParams["lines.linewidth"]-j*0.1)

    # Now plot the highlighted neuron
    for j in range(n_b):
        for i in range(len(i_highlights)):
            li, = ax.plot(t_axis[::plot_skp], hgammaser[::plot_skp, i_highlights[i], j], 
                          color=neuron_colors3[i], ls="-", alpha=1.0-0.1*j, 
                          lw=plt.rcParams["lines.linewidth"]-j*0.1)
            legend_styles[i][j] = li
    
    # Annotations
    ax.set(xlabel="Time (min)", 
           ylabel=r"Alignments $\bar{h}_{i\gamma} = \mathbf{\bar{m}}_i \cdot \mathbf{s}_{\gamma}$")

    fig.tight_layout()
    return fig, ax

In [ ]:
def plot_ibcm_hgammas_matrix(hgammas_mat, i_high, squeeze=0.4):
    n_i, n_comp = hgammas_mat.shape
    neuron_colors3 = neuron_colors_full[[8, 17, 23]]
    
    fig, ax = plt.subplots()
    fig.set_size_inches(plt.rcParams["figure.figsize"][0]*squeeze, 
                        plt.rcParams["figure.figsize"][1])
    # Extent: left, right, bottom, top
    # Greyscale version
    #ax.imshow(hgammas_matrix, cmap="Greys", aspect=0.6, extent=(0.5, n_comp+0.5, 0.5, n_i))
    #ax.set_xticks(list(range(1, n_comp+1)))
    # Colorful version: add patches manually with fill_between. 
    # Color highlighted neurons, leave others grayscale!
    normed_matrix = (hgammas_mat - hgammas_mat.min()) / (hgammas_mat.max() - hgammas_mat.min())
    for i in range(n_i):
        # Full rainbow version
        #cmap = sns.light_palette(neuron_colors_full[i], as_cmap=True)
        # Version where only highlights are colored
        if i in i_high:
            cmap = sns.light_palette(neuron_colors3[i_high.index(i)], as_cmap=True)
        else:
            cmap = sns.color_palette("Greys", as_cmap=True)
        for j in range(n_comp):
            ax.fill_between([-0.5+j, 0.5+j], -0.5+i, 0.5+i, color=cmap(normed_matrix[i, j]))

    ax.set_xlim([-0.6, -0.6+n_comp])
    ax.set_ylim([-0.6, -0.6+n_i])
    ax.set_yticks(list(range(0, 19, 2)) + [21, 23])

    for i, lbl in enumerate(ax.get_yticklabels()):
        if int(lbl.get_text()) in i_high:
            clr = neuron_colors3[i_high.index(int(lbl.get_text()))]
            lbl.set_color(clr)
            ax.yaxis.get_ticklines()[i].set_color(clr)

    ax.set_xlabel(r"Component $\gamma$", size=6)
    ax.set_ylabel(r"IBCM neuron index $i$", size=6)
    cbar = fig.colorbar(mpl.cm.ScalarMappable(
        norm=mpl.colors.Normalize(hgammas_mat.min(), hgammas_mat.max()), 
        cmap="Greys"), ax=ax, aspect=30, pad=0.1)
    cbar.set_ticks([])
    cbar.set_label(label=r"Alignments ${\bar{h}}_{i\gamma}$, 45 min", fontsize=6)
    fig.tight_layout()
    return fig, ax, cbar


# Load IBCM simulations

In [ ]:
def recompute_specifs(hgamser, transient_frac=5/6):
    """ To compute the odor with which each IBCM aligned most. """
    duration_loc = hgamser.shape[0]
    transient_loc = int(transient_frac * duration_loc)
    hgammean = np.mean(hgamser[transient_loc:], axis=0)
    # Sorted odor indices, from min to max, of odor alignments for each neuron
    aligns_idx_sorted = np.argsort(hgammean, axis=1) 
    specifs = np.argmax(hgammean, axis=1)
    return specifs

In [ ]:
# Extract saved simulation
with np.load(pj(data_folder_conv, "ibcm_convergence_hgammas_examples.npz")) as fp:
    hgammas_def = fp["hgammas_ser_def"]
    specifs_def = recompute_specifs(hgammas_def)
    
    hgammas_low = fp["hgammas_ser_low"]
    specifs_low = recompute_specifs(hgammas_low)
    
    hgammas_hi = fp["hgammas_ser_hi"]
    specifs_hi = recompute_specifs(hgammas_hi)

# Panel A: good IBCM example, annotation of convergence metrics

In [ ]:
tu_scale = 1.0 / 100.0 / 60.0  # 10 ms steps to min
n_i_ibcm = hgammas_def.shape[1]
nsteps = hgammas_def.shape[0]
n_components = hgammas_def.shape[2]
tser_example = np.arange(0.0, 360000.0, 360000 / nsteps) * tu_scale  # in min

transient = int(3*tser_example.size/4)
i_highlights = [2, 8, 21]  # Neurons to highlight

hgammas_matrix = np.mean(hgammas_def[transient:], axis=0)

In [ ]:
fig, ax = plot_ibcm_hgammas_series(tser_example, i_highlights, hgammas_def, squeeze=1.0)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.15, plt.rcParams["figure.figsize"][1])
n_odors = hgammas_def.shape[1]

# Extra annotations to explain the convergence metrics
arrowprops = {"arrowstyle": "<->", "lw":0.75, "color":"k"}

# Alignment gaps
ytop = np.mean(hgammas_def[-10000:-1, i_highlights[2], specifs_def[i_highlights[2]]])
ybot = np.mean(hgammas_def[-10000:-1, i_highlights[2], (specifs_def[i_highlights[2]] + 1) % n_odors])
ax.annotate("", xy=(63, ybot), xytext=(63, ytop), arrowprops=arrowprops, annotation_clip=False)
ax.annotate("Alignment\nstrength " + r"$\Delta h$", xy=(70, 0.5*(ybot+ytop)), rotation=90, 
            ha="center", va="center", weight="semibold", annotation_clip=False)

# Variance of h_gammas
hgampad = 3.0
ax.axhline(ytop+hgampad, xmin=0.55, xmax=0.95, ls="--", lw=0.75, color="k")
ax.axhline(ytop-hgampad, xmin=0.55, xmax=0.95, ls="--", lw=0.75, color="k")
arrowprops.update({"lw": 0.5, "arrowstyle":"->", "shrinkB":0.01})
xannot = 59.0
ax.annotate("", xy=(xannot, ytop+hgampad), xytext=(xannot, ytop+hgampad+2), arrowprops=arrowprops)
ax.annotate("", xy=(xannot, ytop-hgampad), xytext=(xannot, ytop-hgampad-2), arrowprops=arrowprops)
ax.annotate(r"$\bar{h}_{\gamma, \mathrm{sp}}$ noise", 
           xy=(xannot-1.0, ytop+hgampad+0.1), ha="right", va="bottom", weight="semibold")

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_hgamma_metrics.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, ax, axi = plot_ibcm_hgammas_matrix(hgammas_matrix, i_highlights)
# Extra annotations
ax.set_xticks([])
arrowprops = {"arrowstyle": "<->", "lw":0.75, "color":"k"}
ax.annotate("", xy=(-1, -1.5), xytext=(3, -1.5), arrowprops=arrowprops, annotation_clip=False)
ax.set_xlabel("Odors covered", color="k", labelpad=6.0, weight="semibold")

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_hgamma_metric_matrix.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel D: illustration for low learning rate

In [ ]:
i_highlights = [0, 4, 16]  # Neurons to highlight. Active ones: 4, 12, 14
fig, ax = plot_ibcm_hgammas_series(tser_example, i_highlights, hgammas_low, squeeze=0.65)

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_low_mu_example.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
hgammas_matrix = np.mean(hgammas_low[transient:], axis=0)
fig, ax, axi = plot_ibcm_hgammas_matrix(hgammas_matrix, i_highlights)

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_low_mu_example_matrix.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel E: illustration for high learning rate

In [ ]:
i_highlights = [18, 20]
tsl = slice(5*len(tser_example)//6, None, 1)
fig, ax = plot_ibcm_hgammas_series(tser_example[tsl], i_highlights, hgammas_hi[tsl], squeeze=0.65)
#ax.annotate("Zoom-in", xy=(tser_example[tsl][0], ax.get_ylim()[1]*0.98), va="top", ha="left", fontsize=6)
t_lower = tser_example[tsl][0]
t_mid = 0.5 * (tser_example[tsl][0] + tser_example[tsl][-1])
y_upper = ax.get_ylim()[1]*0.98
ax.annotate("Stochastic\noscillations", xy=(t_lower-0.2, y_upper), va="top", ha="left", fontsize=5)
ax.annotate("", xytext=(t_mid-0.2, y_upper*0.92), 
            xy=(t_mid+1.5, y_upper*0.75), va="top", ha="left", fontsize=5, 
            arrowprops={"arrowstyle":"->"})
ax.annotate("Some\nalignment", xy=(59.0, y_upper), va="top", ha="center", fontsize=5)
ax.annotate("", xytext=(59.0, y_upper*0.82), 
            xy=(59.0, y_upper*0.3), va="top", ha="left", fontsize=5, 
            arrowprops={"arrowstyle":"->"})
ax.set_xlabel("Time, zoom-in (min)")
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_high_mu_example.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
hgammas_matrix = np.mean(hgammas_hi[transient:], axis=0)
fig, ax, axi = plot_ibcm_hgammas_matrix(hgammas_matrix, i_highlights)

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_high_mu_example_matrix.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel B: alignment gap vs rates

In [ ]:
# Conversion of learning rate into inverse seconds units
dtscale_s = 10.0 / 1000.0  # to convert time step units to seconds
factor_mu_s = 1.0 / dtscale_s

In [ ]:
def compute_odor_coverage(specifs):
    nmu, ntau = specifs.shape[:2]
    cov = np.zeros(specifs.shape[:3])
    for i in range(nmu):
        for j in range(ntau):
            for k in range(specifs.shape[2]):
                cov[i, j, k] = np.unique(specifs[i, j, k]).size
    return cov

In [ ]:
# Load complete simulations run on the cluster
with np.load(pj(data_folder_conv, "convergence_vs_ibcm_rates_results_3odors.npz")) as conv_results:
    mutau_grid = conv_results["mutau_grid"]
    mutau_grid[0] *= factor_mu_s  # convert mu into s^-1
    mutau_grid[1] *= dtscale_s  # convert tau into s
    align_gaps = conv_results["align_gaps"]
    gamma_specifs = conv_results["gamma_specifs"]
    hgamma_varis = conv_results["hgamma_varis"]

In [ ]:
n_mu, n_tau = mutau_grid.shape[1:3]

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau-1, -1, -1):
    gap_mean_line_tau = np.mean(align_gaps[:, j, :], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    gap_std_line_tau = np.std(np.mean(align_gaps[:, j, :], axis=2), axis=1, ddof=1)
    ax.plot(mu_range, gap_mean_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    ax.fill_between(mu_range, gap_mean_line_tau-gap_std_line_tau, 
                   gap_mean_line_tau+gap_std_line_tau, color=colors[j], alpha=0.15)
ax.set(xlabel=r"Learning rate $\mu$ ($\mathrm{s^{-1}}$)", 
       ylabel=r"Alignment strength $\Delta h$" + "\n(larger is better)", xscale="log")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, title=r"$\tau_{\Theta}$ (s)", 
          loc="upper left", frameon=False, fontsize=5, bbox_to_anchor=(0.0, 1.05))

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_3odor_alignment_gap_lineplot.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel C: aligment noise
We define the alignment noise as the ratio between the standard deviation over time of the specific alignment, $\sigma[h_\mathrm{specif}]$, and the average alignment strength $\langle \Delta h \rangle$, each averaged across simulations. This gives a sense of the amplitude of $h$ fluctuations compared to the gap between $h_\mathrm{sp}$ and $h_\mathrm{ns}$. 

In [ ]:
def get_vari_specifs(allhvaris, specifs):
    vari_specifs = np.zeros(allhvaris.shape[:4])
    n_mu, n_tau, n_seed, n_i = specifs.shape[:4]
    for i in range(n_mu):
        for j in range(n_tau):
            for k in range(n_seed):
                vari_specifs[i, j, k] = allhvaris[i, j, k][np.arange(n_i), specifs[i, j, k]]
    return vari_specifs

In [ ]:
fig, ax = plt.subplots()
colors = sns.color_palette("magma", n_colors=n_tau)
# Plot the variance of the specific c_gamma, mean across neurons and seeds
vari_specifs = get_vari_specifs(hgamma_varis, gamma_specifs)

# Square root from variance to standard deviation
# and normalize by the average alignment gap
stdev_specifs = np.sqrt(vari_specifs) / np.mean(align_gaps, axis=(2,3), keepdims=True)

for j in range(n_tau):
    std_mean_line_tau = np.mean(stdev_specifs[:, j, :], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    std_std_line_tau = np.std(stdev_specifs[:, j, :], axis=(1, 2), ddof=1)
    ax.plot(mu_range, std_mean_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    #ax.fill_between(mu_range, std_mean_line_tau-std_std_line_tau, 
    #               std_mean_line_tau+std_std_line_tau, color=colors[j], alpha=0.15)
ax.set(xlabel=r"Learning rate $\mu$ ($\mathrm{s^{-1}}$)", 
       #ylabel=r"Standard dev. of $h_{\gamma,sp}$" + "\n" + r"scaled by average $\Delta h$", 
       ylabel="Alignment noise,\n" + r"$\sigma[h_\mathrm{sp}] / \langle \Delta h \rangle$",
       xscale="log", yscale="log")
ax.legend(title=r"$\tau_{\Theta}$ (s)", ncol=2, frameon=False)

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_3odor_alignment_noise.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel F: alignment strength vs odor number

 Alignment gap, average fraction odors covered as a function of number of odors. 
 
 Pick one $\tau$, show lines for different $\mu$. 
 
 Legend will be shared with the next panel too. 

In [ ]:
concatenated_align_gaps = []
concatenated_odor_coverages = []
concatenated_hgams_varis = []
nodors_range = np.asarray([3, 4, 5, 6, 8])
f_name_prefix = "convergence_vs_ibcm_rates_results_"
for n in nodors_range:
    try:
        fp = np.load(pj(data_folder_conv, f_name_prefix + f"{n}odors.npz"))
    except FileNotFoundError: 
        continue
    else:
        concatenated_align_gaps.append(fp["align_gaps"])
        specifs_n = fp["gamma_specifs"]
        concatenated_odor_coverages.append(compute_odor_coverage(specifs_n))
        vari_specifs = get_vari_specifs(fp["hgamma_varis"], fp["gamma_specifs"])
        concatenated_hgams_varis.append(vari_specifs)
        mutau_grid = fp["mutau_grid"]
        mutau_grid[0] *= factor_mu_s  # convert mu into s^-1
        mutau_grid[1] *= dtscale_s  # convert tau into s
concatenated_align_gaps = np.stack(concatenated_align_gaps)
concatenated_odor_coverages = np.stack(concatenated_odor_coverages)
concatenated_hgams_varis = np.stack(concatenated_hgams_varis)



In [ ]:
chosen_tau_j = 5  # 1600
fig, ax = plt.subplots()
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*0.91, plt.rcParams["figure.figsize"][1])
mu_range = mutau_grid[0, :, 0]
n_mu = mu_range.shape[0]
colors = sns.color_palette("ocean", n_colors=n_mu)
for i in range(n_mu-1, -1, -1):
    gap_mean_line_mu = np.mean(concatenated_align_gaps[:, i, chosen_tau_j], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    gap_std_line_mu = np.std(np.mean(concatenated_align_gaps[:, i, chosen_tau_j], axis=2), axis=1, ddof=1)
    ax.plot(nodors_range, gap_mean_line_mu, color=colors[i], 
            label=str(mutau_grid[0, i, 0]), marker=markerstyles[i], ms=3)
    ax.fill_between(nodors_range, gap_mean_line_mu-gap_std_line_mu, 
                   gap_mean_line_mu+gap_std_line_mu, color=colors[i], alpha=0.1)
ax.set(xlabel=r"# odors $N_\mathrm{B}$ (in $N_\mathrm{S}=25$ dim.)", 
       ylabel="Alignment strength\n(larger is better)")
handles, labels = ax.get_legend_handles_labels()
#ax.legend(handles=handles, labels=labels, 
#          title=r"$\mu$ ($\mathrm{s^{-1}}$)", 
#          loc="upper left", bbox_to_anchor=(0.98, 1), frameon=False)
# Annotate with tau
ax.set_title(r"$\tau_{\Theta} = " + "{:d}$ s".format(int(mutau_grid[1, 0, chosen_tau_j])), y=0.9)

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_alignment_vs_nodors_nolegend.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()


# Panel G: odor coverage vs odor number

In [ ]:
fig, ax = plt.subplots()
# No legend, shared from previous plot
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*0.91, plt.rcParams["figure.figsize"][1])
mu_range = mutau_grid[0, :, 0]
for i in range(n_mu-1, -1, -1):
    coverage_i = concatenated_odor_coverages[:, i, chosen_tau_j]
    coverage_line_mu = np.mean(coverage_i, axis=1) / nodors_range
    coverage_std_mu = np.std(coverage_i, axis=1) / nodors_range  # std across seed
    ax.plot(nodors_range, coverage_line_mu, color=colors[i], 
            label=str(mutau_grid[0, i, 0]), marker=markerstyles[i], ms=3)
    ax.fill_between(nodors_range, coverage_line_mu-coverage_std_mu, 
                   coverage_line_mu+coverage_std_mu, color=colors[i], alpha=0.1)
ax.set(xlabel=r"# odors $N_\mathrm{B}$ (in $N_\mathrm{S}=25$ dim.)", 
       ylabel="Average fraction\nodors covered")
handles, labels = ax.get_legend_handles_labels()
#ax.legend(handles=handles, labels=labels, 
#          title=r"$\mu$ ($\mathrm{s^{-1}}$)", 
#          loc="upper left", bbox_to_anchor=(0.98, 1), frameon=False)
#ax.legend(handles=handles, labels=labels, ncol=3,
#          title=r"$\mu$ ($\mathrm{s^{-1}}$)", 
#          loc="lower center", frameon=False)
#ax.set_ylim([0.4, 1.15])
# Annotate with tau
ax.set_title(r"$\tau_{\Theta} = " + "{:d}$ s".format(int(mutau_grid[1, 0, chosen_tau_j])), y=0.9)


fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_odor_coverage_vs_nodors_nolegend.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel H: alignment noise as a function of odor number

In [ ]:
# Average the standard deviation across seeds and neurons, divide by the alignment gap average
# Axes are indexed [n_odor, n_mu, n_tau, n_seeds, n_neurons, ...]
align_noises_vs_nodor = (np.sqrt(concatenated_hgams_varis)
                         / np.mean(concatenated_align_gaps, axis=3, keepdims=True))

fig, ax = plt.subplots()
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.12, plt.rcParams["figure.figsize"][1])
mu_range = mutau_grid[0, :, 0]
for i in range(n_mu-1, -1, -1):
    std_mean_line_mu = np.mean(align_noises_vs_nodor[:, i, chosen_tau_j], axis=(1, 2))
    std_std_line_mu = np.std(np.mean(align_noises_vs_nodor[:, i, chosen_tau_j], axis=2), axis=1, ddof=1)
    ax.plot(nodors_range, std_mean_line_mu, color=colors[i], 
            label=str(mutau_grid[0, i, 0]), marker=markerstyles[i], ms=3)
    ax.fill_between(nodors_range, std_mean_line_mu-std_std_line_mu, 
                   std_mean_line_mu+gap_std_line_mu, color=colors[i], alpha=0.1)
ax.set(xlabel=r"# odors $N_\mathrm{B}$ (in $N_\mathrm{S}=25$ dim.)", 
       ylabel="Alignment noise", #," + r"$\sigma[h_\mathrm{sp}] / \langle \Delta h \rangle$",
      yscale="log")
ax.legend(handles=handles, labels=labels, 
          title=r"$\mu$ ($\mathrm{s^{-1}}$)", 
          loc="upper left", bbox_to_anchor=(0.98, 1), frameon=False)

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_alignment_noise_vs_nodors.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel F alternate: two combined vertical plots

In [ ]:
chosen_tau_j = 5  # 1600
fig, axes = plt.subplots(2, 1, sharex="col")
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.05, plt.rcParams["figure.figsize"][1]*2.1)
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("ocean", n_colors=n_mu)


# First plot: alignment gap
ax = axes[0]
for i in range(n_mu-1, -1, -1):
    gap_mean_line_mu = np.mean(concatenated_align_gaps[:, i, chosen_tau_j], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    gap_std_line_mu = np.std(np.mean(concatenated_align_gaps[:, i, chosen_tau_j], axis=2), axis=1, ddof=1)
    ax.plot(nodors_range, gap_mean_line_mu, color=colors[i], 
            label=str(mutau_grid[0, i, 0]), marker=markerstyles[i], ms=3)
    ax.fill_between(nodors_range, gap_mean_line_mu-gap_std_line_mu, 
                   gap_mean_line_mu+gap_std_line_mu, color=colors[i], alpha=0.1)
ax.set(#xlabel=r"# odors $N_\mathrm{B}$ (in $N_\mathrm{S}=25$ dim.)", 
       ylabel="Alignment strength\n(larger is better)")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, 
          title=r"$\mu$ ($\mathrm{s^{-1}}$)", 
          loc="upper left", bbox_to_anchor=(0.98, 1), frameon=False)
# Annotate with tau
ax.set_title(r"$\tau_{\Theta} = " + "{:d}$ s".format(int(mutau_grid[1, 0, chosen_tau_j])), y=0.9)


# Second plot: odor coverage
ax = axes[1]
for i in range(n_mu-1, -1, -1):
    coverage_i = concatenated_odor_coverages[:, i, chosen_tau_j]
    coverage_line_mu = np.mean(coverage_i, axis=1) / nodors_range
    coverage_std_mu = np.std(coverage_i, axis=1) / nodors_range  # std across seed
    ax.plot(nodors_range, coverage_line_mu, color=colors[i], 
            label=str(mutau_grid[0, i, 0]), marker=markerstyles[i], ms=3)
    ax.fill_between(nodors_range, coverage_line_mu-coverage_std_mu, 
                   coverage_line_mu+coverage_std_mu, color=colors[i], alpha=0.1)
ax.set(xlabel=r"# odors $N_\mathrm{B}$ (in $N_\mathrm{S}=25$ dim.)", 
       ylabel="Average fraction\nodors covered")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, 
          title=r"$\mu$ ($\mathrm{s^{-1}}$)", 
          loc="upper left", bbox_to_anchor=(0.98, 1), frameon=False)
#ax.legend(handles=handles, labels=labels, ncol=3,
#          title=r"$\mu$ ($\mathrm{s^{-1}}$)", 
#          loc="lower center", frameon=False)
#ax.set_ylim([0.4, 1.15])
# Annotate with tau
#ax.set_title(r"$\tau_{\Theta} = " + "{:d}$ s".format(int(mutau_grid[1, 0, chosen_tau_j])), y=0.9)


fig.tight_layout(h_pad=3.0)
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_ibcm_vs_nodor_combined.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()


# Panel I: compensation between concentration scale and learning rate
Convergence is similar if we keep $\mu c_0^2 = \mathrm{cst}$ as we vary the concentration scale $c_0$. When we also start varying the average conc. by changing time statistics (like whiff and blank concentrations), then we can keep $\mu \langle c \rangle^2 = \mathrm{cst}$ to get approximately the same scaling, although it's not perfect. 

One reason for this is that the convergence time of the first phase scales as $1/(\mu \langle c \rangle^2 + \sigma^2)$, so as $1/(\mu c_0^2)$ (see analytical results on $h_\mathrm{d}$); to keep this time constant, we need to keep $\mu c_0^2 = \mathrm{cst}$. 

The other reason is that the same scaling requirement persists at the fixed point. There, $h_{\mathrm{sp}}$ and $h_{\mathrm{ns}}$ scale as $1/c_0$ (e.g. they have terms like $\langle c \rangle / \sigma^2$, etc.). Since $h = \mathbf{m}^T \mathbf{s} \sim m c_0$, this means that the synaptic weights $ m \sim 1/c_0^2$. Thus, terms in the IBCM equation scale as 

$$ h^2 \mathbf{s} \sim 1/c_0 $$
$$ h \Theta \mathbf{s} \sim h^3 \mathbf{s} \sim 1/c_0^2 $$
$$ -\delta \mathbf{m} \sim 1 / c_0^2 \quad \mathrm{(decay\, term)} $$

so we need to use a learning rate $\mu = \mu_0 c_0^2$ to maintain the same overall scale for the last two terms in the ODE. 


Note also that the convergence is not exactly the same in the Law and Cooper variant, where the effective learning rate is normalized by $k_{\Theta} + \Theta$ and so varies differently, through $\Theta$'s magnitude, depending on $c_0$. 

### Scaling of $h$ with the background magnitude
Importantly, note that $h \sim 1/c_0$ at steady-state, so the alignment gap needs to be looked at relative to the background scale: plot $\Delta h c_0$. 

Another option, more agnostic to $c_0$ directly, would be to scale relative to the analytical prediction $h_{\mathrm{sp}}$, which gives the expected scale of the alignment gap, irrespective of whether the simulations reached it or not. Finding $\Delta h / (h_\mathrm{sp} - h_\mathrm{ns})$ near $1$ would be good, far from 1 would be bad (means it hasn't converged). 

In [ ]:
# Load complete simulations run on the cluster
with np.load(pj(data_folder_conv, "convergence_vs_background_ampli_results_3odors.npz")) as conv_results:
    muc0_grid = conv_results["muc0_grid"]
    muc0_grid[0] *= factor_mu_s  # convert mu into s^-1
    align_gaps_c0 = conv_results["align_gaps"]
    gamma_specifs_c0 = conv_results["gamma_specifs"]
    hgamma_varis_c0 = conv_results["hgamma_varis"]
    
    # Predicted gaps to scale the alignment gap and the variance
    align_gaps_th = conv_results["gaps_th"]
    align_gaps_c0_scaled = align_gaps_c0 / align_gaps_th[:, :, None, None]
    hgamma_varis_c0_scaled = hgamma_varis_c0 / align_gaps_th[:, :, None, None, None]**2.0

In [ ]:
# Simulations with large $\mu$ and large $c_0$ give very large IBCM fluctuations
# as can be checked manually by running simulations with these parameters.
# so their large alignment is an artifact. We filter out seeds (axis 2) 
# where at least one neuron (max along axis 3) have an alignment (axis4) above a threshold. 
# The threshold is a standard deviation in h_gamma larger than 8x the theoretical h gap
# in other words, a scaled variance (hgamma_varis_c0_scaled) larger than 64. 
seeds_excess_variance = hgamma_varis_c0_scaled.max(axis=4).max(axis=3) > 64.0
# Neurons in these fluctuating seeds do not really align -- set their gap to zero
align_gaps_c0_scaled_corrected = np.copy(align_gaps_c0_scaled)
n_i = align_gaps_c0_scaled.shape[3]  # number of neurons
mask = np.tile(seeds_excess_variance[:, :, :, None], (1, 1, 1, n_i))
align_gaps_c0_scaled_corrected[mask] = 0.0

# Print how many seeds in each grid point have large fluctuations 
# and thus artifactual alignment strength
print(seeds_excess_variance.sum(axis=2))  

In [ ]:
n_mu, n_c0 = muc0_grid.shape[1:3]

mu_range = muc0_grid[0, :, 0]
c0_range = muc0_grid[1, 0, :]  # c0 varies along axis 1
# Heatmap
fig, ax = plt.subplots()
im = ax.pcolormesh(np.log10(muc0_grid[1]), np.log10(muc0_grid[0]), 
                   np.mean(align_gaps_c0_scaled_corrected, axis=(2, 3)), 
                   cmap="viridis", vmin=0.0)
ax.set(ylabel=r"$\log_{10}$ learning rate $\mu$", 
       xlabel=r"$\log_{10}$ concentration scale $c_0$")#, yscale="log", xscale="log")
#ax.set_ylim(mu_range[0], mu_range[-1])
#ax.set_xlim(c0_range[0], c0_range[-1])
fig.colorbar(im, label=r"Scaled alignment $\Delta h$")

# Add a theoretical line mu \sim 1/c_0^2
# Reference point is mu = 7.5e-4 * factor_mu_s, c_0 = 0.6
mu_ref = mu_range[2]
c0_ref = c0_range[2]
mu_c0_ref_line = mu_ref * (c0_ref / c0_range) ** 2.0
# Offset this line, multiply c0 by a factor < 0
ax.plot(np.log10(c0_range), np.log10(mu_c0_ref_line), ls="--", color="w")
ax.annotate(r"$\mu c_0^2 = \mathrm{cst}$", xy=(np.log10(c0_ref)-0.2, np.log10(mu_ref)-1.25), 
            rotation=-45, color="w")
ax.annotate("Slow\nconvergence", xy=(np.log10(c0_range[0]), np.log10(mu_range[-1])), 
           ha="left", va="bottom", color="w", fontsize=6)
ax.annotate(r"Large $\bar{h}_{i\gamma}$" + "\nfluctuations", xy=(np.log10(c0_range[-1]), np.log10(mu_range[0])), 
           ha="right", va="top", color="w", fontsize=6)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_backscale_alignment_gap.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel J: as a function of whiff and blank duration
The learning rate is adapted to the average concentration. 

Alignment gap $\Delta h \sim 1/\langle c \rangle$, and $\langle c \rangle \sim \chi$ the probability of a whiff, so we plot $\chi \Delta h$. 

In [ ]:
# Load complete simulations run on the cluster
with np.load(pj(data_folder_conv, "convergence_vs_turbulence_strength_results_3odors.npz")) as conv_results:
    twtb_grid = conv_results["twtb_grid"]
    twtb_grid[0] *= dtscale_s  # convert t_whiffs into s
    twtb_grid[1] *= dtscale_s  # convert t_blanks into s
    align_gaps_twtb = conv_results["align_gaps"]
    gamma_specifs_twtb = conv_results["gamma_specifs"]
    hgamma_varis_twtb = conv_results["hgamma_varis"]
    # Predicted gaps to scale the alignment gap and the variance
    align_gaps_th = conv_results["gaps_th"]
    align_gaps_twtb_scaled = align_gaps_twtb / align_gaps_th[:, :, None, None]
    hgamma_varis_twtb_scaled = hgamma_varis_twtb / align_gaps_th[:, :, None, None, None]**2.0

In [ ]:
n_tw, n_tb = twtb_grid.shape[1:3]

tw_range = twtb_grid[0, :, 0]  # ij meshgrid indexing, tw is rows (y axis)
tb_range = twtb_grid[1, 0, :]
# Heatmap
fig, ax = plt.subplots()
# x axis is t_b, y axis is t_w
im = ax.pcolormesh(np.arange(tb_range.shape[0]), np.arange(tw_range.shape[0]), 
                   np.mean(align_gaps_twtb_scaled, axis=(2, 3)), cmap="viridis", vmin=0.0)
ax.set(xlabel=r"Blank max. duration (s)", ylabel=r"Whiff max. duration (s)")
#ax.set_xlim(tb_range[0], tb_range[-1])
#ax.set_ylim(tw_range[0], tw_range[-1])
ax.set_xticks(np.arange(tw_range.shape[0]))
ax.set_yticks(np.arange(tb_range.shape[0]))
def label_formatter(x):
    if x < 0.1:
        return "{:.2f}".format(x)
    elif x < 1.0:
        return "{:.1f}".format(x)
    else:
        return "{:d}".format(int(x))
ax.set_xticklabels((label_formatter(a) for a in tw_range))
ax.set_yticklabels((label_formatter(a) for a in tb_range))
tau_avg = 2000.0 * dtscale_s
# x coords are 0, ..., 6 for t=2*10^1 * 0.01 s, ..., t=2*10^4 * 0.01 s, etc.
# so convert tau in unit steps to coords as t=2*10^{x/2+1-2} -> x = 2*(log10(t/2) + 1)
def s_to_x(s):
    return (2.0 * (np.log10(s/2) + 1.0))
taux, tauy = s_to_x(tau_avg), s_to_x(tau_avg)
axis_to_data = ax.transAxes + ax.transData.inverted()
data_to_axis = axis_to_data.inverted()
ax.axvline(taux, ymax=data_to_axis.transform((taux, tauy))[1], ls="--", color="w")
ax.axhline(tauy, xmax=data_to_axis.transform((taux, tauy))[0], ls="--", color="w")
ax.annotate(r"$\tau_{\Theta}$", xy=(taux/2, tauy+0.1), ha="center", va="bottom", color="w")
ax.annotate(r"$\tau_{\Theta}$", xy=(taux+0.1, tauy/2), ha="left", va="center", color="w")

fig.colorbar(im, label=r"Scaled alignment $\Delta h$")

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_whiffblanks_duration_alignment_gap.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Last two panels: BioPCA versus turbulence strength

In [ ]:
# Load complete simulations run on the cluster
with np.load(pj(data_folder_conv, "biopca_convergence_vs_background_ampli_3odors.npz")) as conv_results:
    muc0_grid_pca = conv_results["muc0_grid"]
    muc0_grid_pca[0] *= factor_mu_s  # convert mu into s^-1
    true_pvs = conv_results["true_pvs"]
    learn_pvs = conv_results["learn_pvs"]
    vari_pvs = conv_results["vari_pvs"]
    align_errs = conv_results["align_errs"]

In [ ]:
def l2_norm(a, axis=-1):
    return np.sqrt(np.sum(a**2, axis=axis))

In [ ]:
n_mu, n_c0 = muc0_grid_pca.shape[1:3]

mu_range = muc0_grid_pca[0, :, 0]
c0_range = muc0_grid_pca[1, 0, :]  # c0 varies along axis 1
# Heatmap
fig, ax = plt.subplots()
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.05, 
                    plt.rcParams["figure.figsize"][1])

im = ax.pcolormesh(np.log10(muc0_grid_pca[1]), np.log10(muc0_grid_pca[0]), 
                   np.mean(np.log10(align_errs), axis=2), 
                   cmap="viridis_r")

ax.annotate("Slow\nconvergence", xy=(np.log10(c0_range[0]), np.log10(mu_range[0])), 
           ha="left", va="bottom", color="w", fontsize=6)
ax.annotate("Large fluctuations", xy=(0.5*np.log10(c0_range[-1]*c0_range[0]), np.log10(mu_range[-1])), 
           ha="center", va="top", color="w", fontsize=6)
ax.set(ylabel=r"$\log_{10}$ learning rate $\mu$", 
       xlabel=r"$\log_{10}$ concentration scale $c_0$")#, yscale="log", xscale="log")
#ax.set_ylim(mu_range[0], mu_range[-1])
#ax.set_xlim(c0_range[0], c0_range[-1])
fig.colorbar(im, label=r"$\log_{10}$ alignment error" +"\n(brighter is better)")


fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_biopca_backscale_alignerr.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Effect of turbulence strength?

In [ ]:
# Load complete simulations run on the cluster
with np.load(pj(data_folder_conv, "biopca_convergence_vs_turbulence_strength_3odors.npz")) as conv_results:
    twtb_grid = conv_results["twtb_grid"]
    twtb_grid[0] *= dtscale_s  # convert t_whiffs into s
    twtb_grid[1] *= dtscale_s  # convert t_blanks into s
    true_pvs_twtb = conv_results["true_pvs"]
    learn_pvs_twtb = conv_results["learn_pvs"]
    vari_pvs_twtb = conv_results["vari_pvs"]
    align_errs_twtb = conv_results["align_errs"]

In [ ]:
n_tw, n_tb = twtb_grid.shape[1:3]

tw_range = twtb_grid[0, :, 0]  # ij meshgrid indexing, tw is rows (y axis)
tb_range = twtb_grid[1, 0, :]
# Heatmap
fig, ax = plt.subplots()
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.05, 
                    plt.rcParams["figure.figsize"][1])
# x axis is t_b, y axis is t_w

# Alignment error
im = ax.pcolormesh(np.arange(tb_range.shape[0]), np.arange(tw_range.shape[0]), 
                   np.mean(np.log10(align_errs_twtb), axis=2), cmap="viridis_r")

# CV (sigma/mean) in eigenvalues
#im = ax.pcolormesh(np.arange(tb_range.shape[0]), np.arange(tw_range.shape[0]), 
#                   np.mean(np.sqrt(vari_pvs_twtb)/true_pvs_twtb[:, :, :, :3], axis=(2,3)), cmap="viridis_r")

# Difference in eigenvalues
#im = ax.pcolormesh(np.arange(tb_range.shape[0]), np.arange(tw_range.shape[0]), 
#                   np.mean(np.log10(learn_pvs_twtb/true_pvs_twtb[:, :, :, :3]), axis=(2,3)), cmap="viridis")


ax.set(xlabel=r"Blank max. duration (s)", ylabel=r"Whiff max. duration (s)")
#ax.set_xlim(tb_range[0], tb_range[-1])
#ax.set_ylim(tw_range[0], tw_range[-1])
ax.set_xticks(np.arange(tw_range.shape[0]))
ax.set_yticks(np.arange(tb_range.shape[0]))
def label_formatter(x):
    if x < 0.1:
        return "{:.2f}".format(x)
    elif x < 1.0:
        return "{:.1f}".format(x)
    else:
        return "{:d}".format(int(x))
ax.set_xticklabels((label_formatter(a) for a in tw_range))
ax.set_yticklabels((label_formatter(a) for a in tb_range))

fig.colorbar(im, label=r"$\log_{10}$ alignment error" + "\n (brighter is better)")

fig.tight_layout()

# Maybe show in referee response only, since it is similar to IBCM
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_biopca_whiffblanks_alignerr.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Effect of the number of odors on BioPCA?


In [ ]:
concatenated_align_errs = []
concatenated_pvs_diffs = []
concatenated_pvs_cv = []
nodors_range = np.asarray([3, 4, 5, 6, 8])
f_name_prefix = "biopca_convergence_vs_background_ampli_"
for n in nodors_range:
    try:
        fp = np.load(pj(data_folder_conv, f_name_prefix + f"{n}odors.npz"))
    except FileNotFoundError: 
        continue
    else:
        concatenated_align_errs.append(fp["align_errs"])
        pvs_cv_n = np.mean(np.sqrt(fp["vari_pvs"]) / fp["true_pvs"][:, :, :, :n], axis=3)
        concatenated_pvs_cv.append(pvs_cv_n)
        pvs_log_diff = l2_norm(np.log10(fp["learn_pvs"] / fp["true_pvs"][:, :, :, :n]), axis=3)
        concatenated_pvs_diffs.append(pvs_log_diff)
        
concatenated_pvs_cv = np.stack(concatenated_pvs_cv)
concatenated_pvs_diffs = np.stack(concatenated_pvs_diffs)
concatenated_align_errs = np.stack(concatenated_align_errs)

In [ ]:
muc0_grid_pca[1, 0, :]

In [ ]:
chosen_c0_j = 2  # 0.6
fig, ax = plt.subplots()
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.1, plt.rcParams["figure.figsize"][1])
mu_range = muc0_grid_pca[0, :, 0]
n_mu = mu_range.shape[0]
colors = sns.color_palette("ocean", n_colors=n_mu)
for i in range(n_mu-1, -1, -1):
    y = np.log10(concatenated_align_errs[:, i, chosen_c0_j])
    err_mean_line_mu = np.mean(y, axis=1)
    err_std_line_mu = np.std(y, axis=1, ddof=1)
    ax.plot(nodors_range, err_mean_line_mu, color=colors[i], 
            label="{:.2f}".format(muc0_grid_pca[0, i, 0]*60.0), marker=markerstyles[i], ms=3)
    ax.fill_between(nodors_range, err_mean_line_mu-err_std_line_mu, 
                   err_mean_line_mu+err_std_line_mu, color=colors[i], alpha=0.1)
ax.set(xlabel=r"# odors $N_\mathrm{B}$ (in $N_\mathrm{S}=25$ dim.)", 
       ylabel=r"$\log_{10}$ alignment error" + "\n(smaller is better)")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, 
          title=r"$\mu$ ($\mathrm{min^{-1}}$)", 
          loc="upper left", bbox_to_anchor=(0.98, 1), frameon=False)

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_biopca_alignerr_vs_nodors.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()


In [ ]:
chosen_c0_j = 2  # index 0: 0.0375, index 4: 9.6, index 2: 0.6
fig, ax = plt.subplots()
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.1, plt.rcParams["figure.figsize"][1])
mu_range = muc0_grid_pca[0, :, 0]
n_mu = mu_range.shape[0]
colors = sns.color_palette("ocean", n_colors=n_mu)
for i in range(n_mu-1, -1, -1):
    y = np.log10(concatenated_pvs_diffs)
    err_mean_line_mu = np.mean(y[:, i, chosen_c0_j], axis=1)
    err_std_line_mu = np.std(y[:, i, chosen_c0_j], axis=1, ddof=1)
    ax.plot(nodors_range, err_mean_line_mu, color=colors[i], 
            label="{:.2f}".format(muc0_grid_pca[0, i, 0]*60.0), marker=markerstyles[i], ms=3)
    ax.fill_between(nodors_range, err_mean_line_mu-err_std_line_mu, 
                   err_mean_line_mu+err_std_line_mu, color=colors[i], alpha=0.1)
ax.set(xlabel=r"# odors $N_\mathrm{B}$ (in $N_\mathrm{S}=25$ dim.)", 
       ylabel=r"$\log_{10}$ error on PVs" + "\n(smaller is better)")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, 
          title=r"$\mu$ ($\mathrm{min^{-1}}$)", 
          loc="upper left", bbox_to_anchor=(0.98, 1), frameon=False)

fig.tight_layout()
# Maybe show to referees only, since similar to alignment error and avoids introducing
# an extra metric in the manuscript
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_biopca_pvs_error_vs_nodors.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()


In [ ]:
chosen_c0_j = 2  # index 0: 0.0375, index 4: 9.6, index 2: 0.6
fig, ax = plt.subplots()
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.1, plt.rcParams["figure.figsize"][1])
mu_range = muc0_grid_pca[0, :, 0]
n_mu = mu_range.shape[0]
colors = sns.color_palette("ocean", n_colors=n_mu)
for i in range(n_mu-1, -1, -1):
    y = np.log10(concatenated_pvs_cv)
    err_mean_line_mu = np.mean(y[:, i, chosen_c0_j], axis=1)
    err_std_line_mu = np.std(y[:, i, chosen_c0_j], axis=1, ddof=1)
    ax.plot(nodors_range, err_mean_line_mu, color=colors[i], 
            label="{:.2f}".format(muc0_grid_pca[0, i, 0]*60.0), marker=markerstyles[i], ms=3)
    ax.fill_between(nodors_range, err_mean_line_mu-err_std_line_mu, 
                   err_mean_line_mu+err_std_line_mu, color=colors[i], alpha=0.1)
ax.set(xlabel=r"# odors $N_\mathrm{B}$ (in $N_\mathrm{S}=25$ dim.)", 
       ylabel=r"$\log_{10}$ error on PVs" + "\n(smaller is better)")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, 
          title=r"$\mu$ ($\mathrm{min^{-1}}$)", 
          loc="upper left", bbox_to_anchor=(0.98, 1), frameon=False)

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_biopca_pvs_vari_vs_nodors.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()
